In [20]:
import sys
#sys.path.insert(0, '/home/jovyan/work/Molecular_Properties/model-pipeline/src')
import pandas as pd
import warnings
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#from model_construction import prepare_data_split, run_model, show_varimp
from bearinmind_pipeline.data_preprocessing import dataPreprocessing as dp
#from bearinmind_pipeline.model_construction import modelBuilder as mb
import bearinmind_pipeline.model_construction as mc
from numpy import inf
from sklearn.metrics import mean_squared_error
from math import sqrt
import gc
from tqdm import tqdm_notebook, tqdm 

In [57]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
mapping = pd.read_csv('./data/building_metadata.csv')
weather_train = pd.read_csv('./data/weather_train.csv')
weather_test = pd.read_csv('./data/weather_test.csv')

In [58]:
df_train = df_train.merge(mapping, how = "left", on = "building_id")
df_test = df_test.merge(mapping, how = "left", on = "building_id")

In [59]:
df_train = df_train.merge(weather_train, how = "left", on = ["site_id", "timestamp"])
df_test = df_test.merge(weather_test, how = "left", on =  ["site_id", "timestamp"])

In [60]:
del weather_train 
del weather_test

In [61]:
df_train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [15]:
def rmsle(y, yhat):
    score = np.sqrt(np.mean(np.power(np.log(yhat+1)-np.log(y.reset_index(drop=True)+1), 2)))
    return(score)

In [16]:
def rmse(y, yhat):
    score = sqrt(mean_squared_error(y, yhat))
    return(score)

In [64]:
df_train.describe()

,building_id,meter,meter_reading,site_id,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
count,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,8.088455e+06,3.506933e+06,2.011944e+07,1.139074e+07,2.011596e+07,1.646708e+07,1.898443e+07,1.876705e+07,2.007242e+07
mean,7.992780e+02,6.624412e-01,2.117121e+03,7.992232e+00,1.077830e+05,1.968277e+03,4.184848e+00,1.598795e+01,1.900423e+00,7.747429e+00,7.964155e-01,1.016085e+03,1.730151e+02,3.377525e+00
std,4.269133e+02,9.309921e-01,1.532356e+05,5.099060e+00,1.171424e+05,3.020815e+01,4.008277e+00,1.094729e+01,2.402909e+00,1.017867e+01,7.468997e+00,7.060539e+00,1.140574e+02,2.265694e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.830000e+02,1.900000e+03,1.000000e+00,-2.890000e+01,0.000000e+00,-3.500000e+01,-1.000000e+00,9.682000e+02,0.000000e+00,0.000000e+00
25%,3.930000e+02,0.000000e+00,1.830000e+01,3.000000e+00,3.252700e+04,1.951000e+03,1.000000e+00,8.600000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.011600e+03,7.000000e+01,2.100000e+00
50%,8.950000e+02,0.000000e+00,7.877500e+01,9.000000e+00,7.270900e+04,1.969000e+03,3.000000e+00,1.670000e+01,0.000000e+00,8.900000e+00,0.000000e+00,1.016000e+03,1.800000e+02,3.100000e+00
75%,1.179000e+03,1.000000e+00,2.679840e+02,1.300000e+01,1.391130e+05,1.993000e+03,6.000000e+00,2.410000e+01,4.000000e+00,1.610000e+01,0.000000e+00,1.020500e+03,2.800000e+02,4.600000e+00
max,1.448000e+03,3.000000e+00,2.190470e+07,1.500000e+01,8.750000e+05,2.017000e+03,2.600000e+01,4.720000e+01,9.000000e+00,2.610000e+01,3.430000e+02,1.045500e+03,3.600000e+02,1.900000e+01


In [65]:
# Check one building
df_train_build = df_train[(df_train['building_id'] == 46) & (df_train['meter'] == 0)]

In [66]:
df_train_build.head(20)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
45,46,0,2016-01-01 00:00:00,53.2397,0,Retail,9045,2016.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2346,46,0,2016-01-01 01:00:00,53.6492,0,Retail,9045,2016.0,NaN,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
4639,46,0,2016-01-01 02:00:00,54.8778,0,Retail,9045,2016.0,NaN,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
6938,46,0,2016-01-01 03:00:00,53.6492,0,Retail,9045,2016.0,NaN,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
9234,46,0,2016-01-01 04:00:00,53.6492,0,Retail,9045,2016.0,NaN,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6
11530,46,0,2016-01-01 05:00:00,54.4683,0,Retail,9045,2016.0,NaN,19.4,NaN,19.4,0.0,NaN,0.0,0.0
13825,46,0,2016-01-01 06:00:00,53.2397,0,Retail,9045,2016.0,NaN,21.1,6.0,21.1,-1.0,1019.4,0.0,0.0
16118,46,0,2016-01-01 07:00:00,53.2397,0,Retail,9045,2016.0,NaN,21.1,NaN,21.1,0.0,1018.8,210.0,1.5
18412,46,0,2016-01-01 08:00:00,53.2397,0,Retail,9045,2016.0,NaN,20.6,NaN,20.0,0.0,1018.1,0.0,0.0
20706,46,0,2016-01-01 09:00:00,54.0588,0,Retail,9045,2016.0,NaN,21.1,NaN,20.6,0.0,1019.0,290.0,1.5


In [67]:
### Intrapolate cloud coverage
df_train['cloud_coverage'] = df_train.groupby(['building_id', 'meter'])['cloud_coverage'].apply(lambda x : x.interpolate(method = "spline", order = 1, limit_direction = "both"))
df_test['cloud_coverage'] = df_test.groupby(['building_id', 'meter'])['cloud_coverage'].apply(lambda x : x.interpolate(method = "spline", order = 1, limit_direction = "both"))

In [68]:
### Intrapolate air temperature
df_train['air_temperature'] = df_train.groupby(['building_id', 'meter'])['air_temperature'].apply(lambda x : x.interpolate(method = "spline", order = 1, limit_direction = "both"))
df_test['air_temperature'] = df_test.groupby(['building_id', 'meter'])['air_temperature'].apply(lambda x : x.interpolate(method = "spline", order = 1, limit_direction = "both"))

In [62]:
### Lags for cloud coverage and air temperature
for i in range(5):
    df_train[f'cloud_coverage_lag{i+1}'] = df_train.groupby(['building_id', 'meter'])['cloud_coverage'].shift(i+1)
    df_test[f'cloud_coverage_lag{i+1}'] = df_test.groupby(['building_id', 'meter'])['cloud_coverage'].shift(i+1)
    df_train[f'air_temperature_lag{i+1}'] = df_train.groupby(['building_id', 'meter'])['air_temperature'].shift(i+1)
    df_test[f'air_temperature_lag{i+1}'] = df_test.groupby(['building_id', 'meter'])['air_temperature'].shift(i+1)

In [64]:
df_train.dtypes

building_id               int64
meter                     int64
timestamp                object
meter_reading           float64
site_id                   int64
primary_use               int64
square_feet               int64
year_built              float64
floor_count             float64
air_temperature         float64
cloud_coverage          float64
dew_temperature         float64
precip_depth_1_hr       float64
sea_level_pressure      float64
wind_direction          float64
wind_speed              float64
Day                       int64
Hour                      int64
DayOfWeek                 int64
cloud_coverage_lag1     float64
air_temperature_lag1    float64
cloud_coverage_lag2     float64
air_temperature_lag2    float64
cloud_coverage_lag3     float64
air_temperature_lag3    float64
cloud_coverage_lag4     float64
air_temperature_lag4    float64
cloud_coverage_lag5     float64
air_temperature_lag5    float64
dtype: object

In [15]:
#### Take a Sample ####
#df_train = df_train.sample(n=int(df_train.shape[0]*0.1), random_state=1)
#df_test = df_test.sample(n=int(df_test.shape[0]*0.1), random_state=1)

In [16]:
#df_train["Month"] = pd.to_datetime(df_train["timestamp"]).dt.month
#df_test["Month"] = pd.to_datetime(df_test["timestamp"]).dt.month

In [69]:
df_train["Day"] = pd.to_datetime(df_train["timestamp"]).dt.day
df_test["Day"] = pd.to_datetime(df_test["timestamp"]).dt.day

In [70]:
df_train["Hour"] = pd.to_datetime(df_train["timestamp"]).dt.hour
df_test["Hour"] = pd.to_datetime(df_test["timestamp"]).dt.hour

In [87]:
df_train.to_csv("df_train.csv", index = False)
df_test.to_csv("df_test.csv", index = False)

In [4]:
df_train = pd.read_csv('./df_train.csv')
df_test = pd.read_csv('./df_test.csv')

In [6]:
df_train["DayOfWeek"] = pd.to_datetime(df_train["timestamp"]).dt.dayofweek
df_test["DayOfWeek"] = pd.to_datetime(df_test["timestamp"]).dt.dayofweek

In [19]:
#df_train = dp.create_stats_features(df_train, 'dew_temperature', ['site_id', 'Month'])
#df_test = dp.create_stats_features(df_test, 'dew_temperature', ['site_id', 'Month'])

In [20]:
#df_train["air_temperature_log"] = np.log(df_train['air_temperature'])
#df_test["air_temperature_log"] = np.log(df_test['air_temperature'])

In [21]:
#df_train["Age"] = pd.to_datetime(df_train["timestamp"]).dt.year - df_train["year_built"]
#df_test["Age"] = pd.to_datetime(df_test["timestamp"]).dt.year - df_test["year_built"]

In [7]:
df_train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,Day,Hour,DayOfWeek
0,0,0,2016-01-01 00:00:00,0.0,0,0,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,4
1,1,0,2016-01-01 00:00:00,0.0,0,0,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,4
2,2,0,2016-01-01 00:00:00,0.0,0,0,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,4
3,3,0,2016-01-01 00:00:00,0.0,0,0,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,4
4,4,0,2016-01-01 00:00:00,0.0,0,0,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,4


In [8]:
#df_train = df_train[df_train["meter_reading"] != 0.0]

In [9]:
df_train.shape

(20216100, 19)

In [10]:
#encode_cols = ["primary_use"]

In [11]:
#data = dp.encode_categorical(df_train, df_test, encode_cols) 

colums primary_use is being encoded


In [12]:
#mb = mc.modelBuilder('regression', 'LGBM')

In [99]:
#data = mb.prepare_data_split(df_train =  data['df_train'], df_test = data['df_test'], 
#                             target = 'meter_reading', rem_cols = ['timestamp'], 
#                             useVarImp = False, varimp_threshold = 100)

17 features have been chosen for modeling


In [100]:
#data['X_test'] = data['X_test'].drop(["row_id"],  axis=1)

In [101]:
#data['Y'] = np.log(data['Y'])

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [102]:
#data['Y'][data['Y'] == -inf] = 0

In [1]:
#data['Y'].describe()

NameError: name 'data' is not defined

In [65]:
# LGBM
params = {
    'boosting_type':'gbdt',
    'n_estimators':500,
    'num_leaves':64, 
    'objective':'regression_l2', 
    'colsample_bytree': 1.0, 
    'learning_rate': 0.3,
    'max_bin':255, 
    'max_depth':9, 
    'metric':'rmse', 
    'min_child_samples':10,
    'min_child_weight':5, 
    'min_split_gain':0.0,
    'random_state':1234,
    'lambda_l2': 1.5,
    'verbose' : 2,
    #'reg_alpha':0.0,
    #'reg_lambda':0.0, 
    #'silent':True, 
    'subsample':1.0
    #,
    #'early_stopping_round' : 100
}

In [66]:
#gc.collect()

In [67]:
#df_train_build = df_train[(df_train['building_id'] == 46) & (df_train['meter'] == 0)]

In [68]:
#df_train_build.head(20)

In [69]:
#submit = mb.run_model(data, n_folds = 5, metric_func = rmse, get_probab = False, 
#                      save_varimp = True, params = params, oversmp = False, cv_shuffle=False)

In [ ]:
### Per meter type ###
perf_dict = {}
final_sub_lst = []
encode_cols = ["primary_use"]
mb = mc.modelBuilder('regression', 'LGBM')
for i in tqdm_notebook(df_train['site_id'].unique()):
#for i in tqdm_notebook(range(2)):
    print(f'training site_id {i}')
    
    df_train_metr = df_train[df_train['site_id'] == i]
    df_test_metr = df_test[df_test['site_id'] == i]
    
    data = dp.encode_categorical(df_train_metr, df_test_metr, encode_cols)
    data = mb.prepare_data_split(df_train =  data['df_train'], df_test = data['df_test'], 
                             target = 'meter_reading', rem_cols = ['timestamp'], 
                             useVarImp = False, varimp_threshold = 100)
    data['X_test'] = data['X_test'].drop(["row_id"],  axis=1)
    data['Y'] = np.log(data['Y'])
    data['Y'][data['Y'] == -inf] = 0
    submit = mb.run_model(data, n_folds = 5, metric_func = rmse, get_probab = False, 
                      save_varimp = False, params = params, oversmp = False, cv_shuffle=False)
    submit['FinalPrediction'] = np.exp(submit['FinalPrediction'])
    final_submit = pd.concat([df_test_metr['row_id'].reset_index(drop=True), pd.DataFrame(submit['FinalPrediction'], columns = ["meter_reading"])], axis = 1)
    
    perf_dict[i] = submit['AverageScore']
    final_sub_lst.append(final_submit)
    

training site_id 0
colums primary_use is being encoded


/opt/conda/lib/python3.7/site-packages/bearinmind_pipeline/data_preprocessing.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[f] = lbl.transform(list(df_train[f].values))
/opt/conda/lib/python3.7/site-packages/bearinmind_pipeline/data_preprocessing.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[f] = lbl.transform(list(df_test[f].values))


27 features have been chosen for modeling


/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Fold: 0
The valuation metric for the fold 0 is 3.156803437950247
Fold: 1
The valuation metric for the fold 1 is 3.2553795950717586
Fold: 2
The valuation metric for the fold 2 is 1.1826569371798568
Fold: 3
The valuation metric for the fold 3 is 1.875100208652426
Fold: 4
The valuation metric for the fold 4 is 4.291632628469619
The average score accross the folds is 2.7523145614647815
training site_id 1
colums primary_use is being encoded


...

Fold: 0
The valuation metric for the fold 0 is 1.611794912200864
Fold: 1
The valuation metric for the fold 1 is 1.60818709189315
Fold: 2
The valuation metric for the fold 2 is 1.3670096781991772
Fold: 3
The valuation metric for the fold 3 is 1.9457789314863347
Fold: 4
The valuation metric for the fold 4 is 1.7171055132724211
The average score accross the folds is 1.6499752254103897
training site_id 15
colums primary_use is being encoded


/opt/conda/lib/python3.7/site-packages/bearinmind_pipeline/data_preprocessing.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[f] = lbl.transform(list(df_train[f].values))
/opt/conda/lib/python3.7/site-packages/bearinmind_pipeline/data_preprocessing.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[f] = lbl.transform(list(df_test[f].values))


27 features have been chosen for modeling


/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Fold: 0


In [ ]:
np.array(list(perf_dict.values())).mean()

In [ ]:
sub = pd.concat(final_sub_lst, axis = 0).reset_index(drop = True)

In [ ]:
sub.shape

In [59]:
sub.to_csv("submit_lightgbm_by_site.csv.gz", index = False, compression='gzip')

In [ ]:
#submit['FinalPrediction'] = np.exp(submit['FinalPrediction'])

In [ ]:
#submit['FinalPrediction']

In [ ]:
#submit

In [ ]:
#final_submit = pd.concat([df_test['row_id'], pd.DataFrame(submit['FinalPrediction'], columns = ["meter_reading"])], axis = 1)

In [ ]:
#final_submit["meter_reading"].describe()

In [ ]:
#final_submit.to_csv("submit_lightgbm_default.csv.gz", index = False, compression='gzip')

In [61]:
!kaggle competitions submit -c ashrae-energy-prediction -f submit_lightgbm_by_site.csv.gz -m "with intrapol and separate model for each site"

100%|████████████████████████████████████████| 472M/472M [00:11<00:00, 42.8MB/s]
Successfully submitted to ASHRAE - Great Energy Predictor III